In [135]:
import time
import pygame as py
import copy
import math
import random
import pickle

# Store Endings

In [136]:
class MyClass():
    def __init__(self, param):
        self.param = param
 
def save_object(obj):
    try:
        with open("data.pickle", "wb") as f:
            pickle.dump(obj, f, protocol=pickle.HIGHEST_PROTOCOL)
    except Exception as ex:
        print("Error during pickling object (Possibly unsupported):", ex)

def load_object(filename):
    try:
        with open(filename, "rb") as f:
            return pickle.load(f)
    except Exception as ex:
        print("Error during unpickling object (Possibly unsupported):", ex)
 


# Trophy

In [137]:
def Trophy(width, height, font, player_hp, colours):
    running = True
    jumping = False
    player_pos = py.Vector2(width/2, 24*height/25)
    dt = 0
    
    game_modes = ["You Died", "Power of Friendship", "At What Cost...", "Civil War"]
    alive = 0
    dead = 0
    
    try:
        endings = load_object("data.pickle")
        
    except Exception as e:
        endings = MyClass([False, False, False, False])

    for x in colours:
        if colours[x] == True:
            alive += 1

        elif colours[x] == False:
            dead += 1
            
            
    if player_hp == 0:
        endings.param[0] = True
        save_object(endings)
        
    elif alive == 6:
        endings.param[1] = True
        save_object(endings)
        
    elif dead == 6:
        endings.param[2] = True
        save_object(endings)
        
    else:
        endings.param[3] = True
        save_object(endings)
        
    
    while running:
        # Closes Program
        for event in py.event.get():
            if event.type == py.QUIT:
                running = False
            
        # Creating Visuals
        screen.fill("white")
        py.draw.circle(screen, "indigo", player_pos, height/23, 0)
        py.draw.circle(screen, "black", player_pos, height/23, 3)
        
        # Health Bar
        advice = font.render('health: '+ str(player_hp) +'/5', True, 'black')
        screen.blit(advice, (2*width/5, height/16))
        
        # Exit
        advice = font.render('Exit', True, 'black')
        screen.blit(advice, (9*width/10, 13*height/16))
        
        # Play Again
        advice = font.render('Play Again', True, 'black')
        screen.blit(advice, (width/10, 13*height/16))
        
        # Statistic Reset
        reset = font.render('Reset', True, 'black')
        screen.blit(reset, (2*width/5, 5*height/16))
        
        # Platform
        py.draw.rect(screen, "brown", (3*width/8, 23*height/35, width/4, height/35))
        py.draw.rect(screen, "black", (3*width/8, 23*height/35, width/4, height/35), 3)
        
        
        # Display Board
        location = 0
            
        hor = width/15
        while hor <= width:

            # Vertical Placement
            vert = height/10
            while vert <= height:

                # Display Found Ending
                if endings.param[location] == True:
                    py.draw.rect(screen, "white", [hor, vert, 3*width/10, height/7])
                    py.draw.rect(screen, "black", [hor, vert, 3*width/10, height/7], 2)
                    mode = font.render(game_modes[location], True, 'black')
                    screen.blit(mode, (hor + 10, vert + 20))

                # Ending not Found
                else:
                    py.draw.rect(screen, "gray", [hor, vert, 3*width/10, height/7])

                location += 1
                vert += height/2 
            hor += 3*width/5
            
                
        # Player on platforms
        if 24*height/25 <= player_pos.y <= height:
            jumping = False
            
            
        elif 23*height/35 <= player_pos.y + height/23 <= 24*height/35 <= height and 3*width/8 <= player_pos.x <= 5*width/8:
            jumping = False


        # Fell off a platform
        elif jumping == False:
            jumping = True
            velocity = 0
            
        keys = py.key.get_pressed()
        # Player Jumps
        
        if keys[py.K_w] and jumping == False:
            jumping = True
            player_pos.y -= height/16
            velocity = 3*height/5

            # Fixes a Teleportation Glitch
            dt = clock.tick(60) / 1000
            dt = 0    
        
        # Circle is Being Pulled by Gravity
        if jumping and player_pos.y <= height:
            player_pos.y -= (velocity*dt) + (-200*dt**2)
            velocity = velocity + (-400*dt)
            

        # Player Moves Horizontally
        if keys[py.K_a] and player_pos.x > height/25:
            player_pos.x -= 300 * dt
            
        if keys[py.K_d] and player_pos.x < width-height/25:
            player_pos.x += 300 * dt
            
            
        if player_pos.x <= width/20:
            running = Start()
            
        elif player_pos.x >= 19*width/20:
            running = False
            
        # Jumping Up
        elif player_pos.y <= 2*height/5:
            # Resets Statistics
            
            endings = MyClass([False, False, False, False])
            save_object(endings)
            
            
        py.display.flip()
        
        # limits FPS to 60
        # dt is delta time in seconds since last frame, used for framerate
        dt = clock.tick(60) / 1000
        


    return(running)

# Voilent Voilet

In [138]:
def Violet(width, height, coins, font, stick, player_hp, colours, string, arrow, bow, dagger):
    
    # Collection of Variables
    dt = 0
    running = True
    player_pos = py.Vector2(width/8, 24*height/25)
    jumping = False
    attack = False
    
    # Attack Timer
    timer = 0
    violet_hp = 10
    spare = False
    front = 3*width/4
    second = 0
    
    
    # violet Dialogue
    confront = ['*The square is determined*', '*The square is focused*', '*violet falters*', 'violet looks conflicted', 'violet relents']
    stages = -1
    
    while running:
        # Closes Program
        for event in py.event.get():
            if event.type == py.QUIT:
                return False

        # Player Options
        hints = 'PRESS 1 [ignore], PRESS 2 [punch]'
        
        if stick == True:
            hints += ', PRESS 3 [stick]'
            
        # Generates inventory
        items = ''
        
        # Player's coins
        if coins != 0:
            items += (str(coins) + ' coin')
        
        if stick:
            items += (', stick')
            
        if string:
            items += (', string')
            
        if arrow:
            items += (', arrow')
            
        if dagger:
            items += (', dagger')
            
        if bow:
            items += (', bow and arrow')
        
        # Creating Background
        screen.fill("white")
        
        # Draws Player
        py.draw.circle(screen, "indigo", player_pos, height/23)
        py.draw.circle(screen, "black", player_pos, height/23, 3)
        
        # Draws violet Square
        if violet_hp >= 4 and stages <= 4 or spare:
            py.draw.rect(screen, "violet", (7*width/8, 11*height/12, height/12, height/12))
            py.draw.rect(screen, "black", (7*width/8, 11*height/12, height/12, height/12), 3)


        keys = py.key.get_pressed()
        
        # Player Jumps
        if keys[py.K_w] and player_pos.y == 24*height/25:

            jumping = True
            # Fixes a Teleportation Glitch
            dt = clock.tick(60) / 1000
            dt = 0
            
        # Circle is Being Pulled by Gravity
        if jumping and player_pos.y <= 24*height/25:
            player_pos.y -= (velocity*dt) + (-200*dt**2)
            velocity = velocity + (-400*dt)
        
        # Circle has Landed
        else:
            jumping = False
            player_pos.y = 24*height/25
            velocity = 3*height/5
            

        # Player Moves Horizontally
        if keys[py.K_a] and player_pos.x > 2*height/25:
            player_pos.x -= 300 * dt
            
        if keys[py.K_d]:
            if attack or violet_hp != 0 and stages != 5:
                if player_pos.x < 3*width/4:
                    player_pos.x += 300 * dt
                    
                    
            elif player_pos.x < width-height/25:
                player_pos.x += 300 * dt

        
        
        if stages < 5 and violet_hp > 5 and attack == False:
            hint = font.render(hints, True, 'black')
            screen.blit(hint, (width/8, height/4))

            # Calm violet
            if keys[py.K_1] and stages <= 4:
                stages += 1
                attack = True
                direction = random.randint(1, 2)

    
            # Punches violet
            elif keys[py.K_2] and violet_hp > 5:
                attack = True
                violet_hp -= 1
                direction = random.randint(1, 2)


            # Uses stick
            elif keys[py.K_3] and stick:
                stick = False
                violet_hp -= 4
                attack = True
                direction = random.randint(1, 2)
                
        # Uses bow and arrow
        if bow and violet_hp <= 5:
            final = font.render('PRESS 4 [bow and arrow]', True, 'black')
            screen.blit(final, (width/8, height/4))
            
            if keys[py.K_4]:
                bow = False
                violet_hp = 0
                
        # Uses dagger
        if dagger and stages == 4:
            mercy = font.render('PRESS 5 [dagger]', True, 'black')
            screen.blit(mercy, (width/8, height/4))
            
            
            if keys[py.K_5]:
                dagger = False
                spare = True
                stages += 1

                
        # Violet Attacks
        
        if attack == True:
            
            if timer <= 3.5:

                if direction == 1:
                    top = height/2
                    py.draw.rect(screen, 'pink', (front, top, width/8, height/8))
                    py.draw.rect(screen, 'pink', (second, top, width/8, height/8))


                if direction == 2:
                    top = 7*height/8
                    py.draw.rect(screen, 'pink', (front, top, width/8, height/8))
                    py.draw.rect(screen, 'pink', (second, top, width/8, height/8))
                    
                py.draw.rect(screen, "pink", (0, 34*height/35, width, height/35))


            else:
                if 7*height/8 <= top:
                    change = -5
                    
                elif height/2 >= top:
                    change = 5
                    
                top += change
                second += 5
                front -= 5

                py.draw.rect(screen, 'violet', (front, top, width/8, height/8))
                py.draw.rect(screen, 'violet', (second, top, width/8, height/8))
                if front - width/23 <= player_pos.x <= front + width/8 + width/23 and top - height/23 <= player_pos.y <= top + height/23 + height/8:
                    player_hp -= 1
                    front = -1*width
                    
                if second - width/23 <= player_pos.x <= second + width/8 + width/23 and top - height/23 <= player_pos.y <= top + height/23 + height/8:
                    player_hp -= 1
                    front = -1*width
                 
                if timer <= 7:
                    py.draw.rect(screen, "violet", (0, 34*height/35, width, height/35))
                    if player_pos.y == 24*height/25:
                        player_hp -= 1
                        attack = False
                        front = -1*width
                        
                        
            timer += 0.06
        
        if front + width/8 < 0:
            attack = False
            timer = 0
            front = 3*width/4
            second = 0
                
            
            
        # Player Health Bar
        advice = font.render('health: ' + str(player_hp) + '/5', True, 'black')
        screen.blit(advice, (width/16, height/16))
        
        # Player Items
        advice = font.render(items, True, 'black')
        screen.blit(advice, (width/16, 3*height/16))
        
        # Enemy Health Bar
        inventory = font.render('health: ' + str(violet_hp) + '/10', True, 'black')
        screen.blit(inventory, (13*width/16, height/16))
        
        # Updates comfort text
        if 0 <= stages <= 4:
            advice = font.render(confront[stages], True, 'black')
            screen.blit(advice, (width/8, height/2))
            
        
        # Updates Screen
        py.display.flip()
        
        # limits FPS to 60
        # dt is delta time in seconds since last frame, used for framerate
        dt = clock.tick(60) / 1000
        
        if violet_hp == 0:
            death = font.render('The sqaure began to flee; you shot an arrow through them', True, 'black')
            screen.blit(death, (width/16, height/4))
            colours['violet'] = False
            
        elif spare:
            mercy = font.render('Violet realized violence was not the answer', True, 'black')
            screen.blit(mercy, (width/16, height/4))
            
        elif stages >= 5 or player_hp <= 5:
            flee = font.render('The sqaure may have been outmatched, but they will return', True, 'black')
            screen.blit(flee, (width/16, height/4))
        
        
        if player_pos.x >= width-height/20 and attack == False:
            running = Trophy(width, height, font, player_hp, colours)

    
    return(running)

# Red

In [139]:
def Red(width, height, coins, font, stick, player_hp, colours, string, arrow, bow, dagger):
    
    # Collection of Variables
    dt = 0
    running = True
    player_pos = py.Vector2(width/8, 24*height/25)
    jumping = False
    attack = False
    
    # Attack Timer
    timer = 0
    red_hp = 5
    top = 0
    
    
    # red Dialogue
    negotiate = ['*The square is fuming*', '*The square continues*', '*Red looks upset*', 'Red is frustrated', 'Red gives up  (You may now pass)']
    stages = -1
    
    while running:
        # Closes Program
        for event in py.event.get():
            if event.type == py.QUIT:
                return False

        # Player Options
        hints = 'PRESS 1 [ignore], PRESS 2 [punch]'
        
        if stick == True:
            hints += ', PRESS 3 [stick]'
            
        if bow == True:
            hints += ', PRESS 4 [bow and arrow]'
            
        if dagger == True:
            hints += ', PRESS 5 [dagger]'
            
        # Generates inventory
        items = ''
        
        # Player's coins
        if coins != 0:
            items += (str(coins) + ' coin')
        
        if stick:
            items += (', stick')
            
        if string:
            items += (', string')
            
        if arrow:
            items += (', arrow')
            
        if dagger:
            items += (', dagger')
            
        if bow:
            items += (', bow and arrow')
        
        # Creating Background
        screen.fill("white")
        
        # Draws Player
        py.draw.circle(screen, "indigo", player_pos, height/23)
        py.draw.circle(screen, "black", player_pos, height/23, 3)
        
        # Draws red Square
        if red_hp != 0:
            py.draw.rect(screen, "red", (7*width/8, 11*height/12, height/12, height/12))
            py.draw.rect(screen, "black", (7*width/8, 11*height/12, height/12, height/12), 3)


        keys = py.key.get_pressed()
        
        # Player Jumps
        if keys[py.K_w] and player_pos.y == 24*height/25:

            jumping = True
            # Fixes a Teleportation Glitch
            dt = clock.tick(60) / 1000
            dt = 0
            
        # Circle is Being Pulled by Gravity
        if jumping and player_pos.y <= 24*height/25:
            player_pos.y -= (velocity*dt) + (-200*dt**2)
            velocity = velocity + (-400*dt)
        
        # Circle has Landed
        else:
            jumping = False
            player_pos.y = 24*height/25
            velocity = 3*height/5
            

        # Player Moves Horizontally
        if keys[py.K_a] and player_pos.x > 2*height/25:
            player_pos.x -= 300 * dt
            
        if keys[py.K_d]:
            if attack or red_hp != 0 and stages != 4:
                if player_pos.x < 3*width/4:
                    player_pos.x += 300 * dt
                    
                    
            elif player_pos.x < width-height/25:
                player_pos.x += 300 * dt

        
        
        if stages < 4 and red_hp != 0 and attack == False:
            hint = font.render(hints, True, 'black')
            screen.blit(hint, (width/8, height/4))

            # Calm Red
            if keys[py.K_1]:
                stages += 1
                attack = True
                spikes = random.sample(range(1, 4), k=3)
                bottom = height/8
                top = 0

    
            # Punches red
            elif keys[py.K_2]:
                attack = True
                red_hp -= 1
                spikes = random.sample(range(1, 4), k=3)
                bottom = height/8
                top = 0


            # Uses stick
            elif keys[py.K_3] and stick:
                stick = False
                red_hp = 0
                
            # Uses bow and arrow
            elif keys[py.K_4] and bow:
                bow = False
                red_hp = 0
                
            # Uses dagger
            elif keys[py.K_5] and dagger:
                dagger = False
                red_hp = 0
                
        # red Attacks
        
        if attack == True:
            
            if timer <= 4:

                if spikes[0] == 1 or spikes[1] == 1:
                    py.draw.polygon(screen, 'pink', [(0, height), (width/32, 7*height/8), (7*width/32, 7*height/8), (width/4, height)])

                if spikes[0] == 2 or spikes[1] == 2:
                    py.draw.polygon(screen, 'pink', [(width/4, height), (9*width/32, 7*height/8), (15*width/32, 7*height/8), (width/2, height)])

                if spikes[0] == 3 or spikes[1] == 3:
                    py.draw.polygon(screen, 'pink', [(width/2, height), (17*width/32, 7*height/8), (23*width/32, 7*height/8), (3*width/4, height)])


            else:
                if spikes[0] == 1 or spikes[1] == 1:
                    py.draw.polygon(screen, 'red', [(0, height), (width/32, 7*height/8), (7*width/32, 7*height/8), (width/4, height)])
                    if 0 <= player_pos.x <= width/4 and 7*height/8 - height/23 <= player_pos.y <= height:
                        player_hp -= 1
                        top = height + 10

                if spikes[0] == 2 or spikes[1] == 2:
                    py.draw.polygon(screen, 'red', [(width/4, height), (9*width/32, 7*height/8), (15*width/32, 7*height/8), (width/2, height)])
                    if width/4 <= player_pos.x <= width/2 and 7*height/8 - height/23 <= player_pos.y <= height:
                        player_hp -= 1
                        top = height + 10

                        
                if spikes[0] == 3 or spikes[1] == 3:
                    py.draw.polygon(screen, 'red', [(width/2, height), (17*width/32, 7*height/8), (23*width/32, 7*height/8), (3*width/4, height)])
                    if width/2 <= player_pos.x <= 3*width/4 and 7*height/8 - height/23 <= player_pos.y <= height:
                        player_hp -= 1
                        top = height + 10
                        
                        
                        
            if timer <= 4:

                if spikes[2] == 1:
                    py.draw.polygon(screen, 'pink', [(0, 0), (width/32, height/8), (7*width/32, height/8), (width/4, 0)])

                if spikes[2] == 2:
                    py.draw.polygon(screen, 'pink', [(width/4, 0), (9*width/32, height/8), (15*width/32, height/8), (width/2, 0)])

                if spikes[2] == 3:
                    py.draw.polygon(screen, 'pink', [(width/2, 0), (17*width/32, height/8), (23*width/32, height/8), (3*width/4, 0)])


            else:
                bottom += 5
                top += 5
                if spikes[2] == 1:
                    py.draw.polygon(screen, 'red', [(0, top), (width/32, bottom), (7*width/32, bottom), (width/4, top)])
                    if 0 <= player_pos.x <= width/4 and top <= player_pos.y <= bottom + height/23:
                        player_hp -= 1
                        top = height + 10
                        

                if spikes[2] == 2:
                    py.draw.polygon(screen, 'red', [(width/4, top), (9*width/32, bottom), (15*width/32, bottom), (width/2, top)])
                    if width/4 <= player_pos.x <= width/2 and top <= player_pos.y <= bottom + height/23:
                        player_hp -= 1
                        top = height + 10
                        
                if spikes[2] == 3:
                    py.draw.polygon(screen, 'red', [(width/2, top), (17*width/32, bottom), (23*width/32, bottom), (3*width/4, top)])
                    if width/2 <= player_pos.x <= 3*width/4 and top <= player_pos.y <= bottom + height/23:
                        player_hp -= 1
                        top = height + 10
                        
            timer += 0.06
        
        if top > height:
            attack = False
            timer = 0
            bottom = height/8
            top = 0
                
            
            
        # Player Health Bar
        advice = font.render('health: ' + str(player_hp) + '/5', True, 'black')
        screen.blit(advice, (width/16, height/16))
        
        # Player Items
        advice = font.render(items, True, 'black')
        screen.blit(advice, (width/16, 3*height/16))
        
        # Enemy Health Bar
        inventory = font.render('health: ' + str(red_hp) + '/5', True, 'black')
        screen.blit(inventory, (13*width/16, height/16))
        
        # Updates comfort text
        if 0 <= stages <= 2:
            advice = font.render(negotiate[stages], True, 'black')
            screen.blit(advice, (width/8, height/2))
            
        
        # Updates Screen
        py.display.flip()
        
        # limits FPS to 60
        # dt is delta time in seconds since last frame, used for framerate
        dt = clock.tick(60) / 1000
        
        if red_hp == 0:
            colours['red'] = False
        
        
        if player_pos.x >= width-height/20 and attack == False:
            running = Violet(width, height, coins, font, stick, player_hp, colours, string, arrow, bow, dagger)

    
    return(running)

# Yellow

In [140]:
def Yellow(width, height, coins, font, stick, player_hp, colours):

    
    # Collection of Variables
    dt = 0
    running = True
    player_pos = py.Vector2(width/8, 24*height/25)
    jumping = False
    attack = False
    shopping = True
    
    # Shop
    buy1 = font.render('PRESS 1 [String - 1 coin], PRESS 2 [Arrow - 2 coins]', True, 'black')
    buy2 = font.render('PRESS 3 [Dagger - 3 coins], PRESS 4 [engage]', True, 'black')
    string = False
    arrow = False
    dagger = False
    bow = False
    
    # Attack Timer
    timer = 0
    yellow_hp = 2
    
    
    # yellow Dialogue
    negotiate = ['*The square looks surpised*', '*Yellow is impressed*', '*Yellow respects your tactics*  (You may now pass)']
    stages = -1
    
    while running:
        # Closes Program
        for event in py.event.get():
            if event.type == py.QUIT:
                return False
            
        # Player Options
        hints = 'PRESS 1 [barter], PRESS 2 [punch]'
        if stick == True:
            hints += ', PRESS 3 [stick]'
            
        if bow == True:
            hints += ', PRESS 4 [bow and arrow]'
            
        if dagger == True:
            hints += ', PRESS 5 [dagger]'
            
        
            
        # Creates bow
        if stick and string and arrow:
            stick = False
            string = False
            arrow = False
            bow = True
            
        # Generates inventory
        items = ''
        
        # Player's coins
        if coins != 0:
            items += (str(coins) + ' coin')
        
        if stick:
            items += (', stick')
            
        if string:
            items += (', string')
            
        if arrow:
            items += (', arrow')
            
        if dagger:
            items += (', dagger')
            
        if bow:
            items += (', bow and arrow')
        
        # Creating Background
        screen.fill("white")
        
        # Draws Player
        py.draw.circle(screen, "indigo", player_pos, height/23)
        py.draw.circle(screen, "black", player_pos, height/23, 3)
        
        # Draws yellow Square
        if yellow_hp != 0:
            py.draw.rect(screen, "yellow", (7*width/8, 11*height/12, height/12, height/12))
            py.draw.rect(screen, "black", (7*width/8, 11*height/12, height/12, height/12), 3)


        keys = py.key.get_pressed()
        
        # Player Jumps
        if keys[py.K_w] and player_pos.y == 24*height/25:

            jumping = True
            # Fixes a Teleportation Glitch
            dt = clock.tick(60) / 1000
            dt = 0
            
        # Circle is Being Pulled by Gravity
        if jumping and player_pos.y <= 24*height/25:
            player_pos.y -= (velocity*dt) + (-200*dt**2)
            velocity = velocity + (-400*dt)
        
        # Circle has Landed
        else:
            jumping = False
            player_pos.y = 24*height/25
            velocity = 3*height/5
            

        # Player Moves Horizontally
        if keys[py.K_a] and player_pos.x > 2*height/25:
            player_pos.x -= 300 * dt
            
        if keys[py.K_d]:
            if attack or yellow_hp != 0 and stages != 2:
                if player_pos.x < 13*width/20:
                    player_pos.x += 300 * dt
                    
                    
            elif player_pos.x < width-height/25:
                player_pos.x += 300 * dt

            
        # Player Engages With yellow Square
        if shopping:
            screen.blit(buy1, (width/10, height/4))
            screen.blit(buy2, (width/10, height/3))
            
            # Buys String
            if keys[py.K_1] and coins >= 1 and string == False:
                coins -= 1
                string = True



            # Buy Arrow
            elif keys[py.K_2] and coins >= 2 and arrow == False:
                coins -= 2
                arrow = True



            # Buy dagger
            elif keys[py.K_3] and stages >= 3 and dagger == False:
                coins -= 3
                dagger = True
                
            elif keys[py.K_4]:
                shopping = False
                attack = True

        
        
        elif stages < 2 and yellow_hp != 0 and attack == False:
            hint = font.render(hints, True, 'black')
            screen.blit(hint, (width/8, height/4))

            # negotiate with Yellow
            if keys[py.K_1]:
                stages += 1
                attack = True
                


            # Punches yellow
            elif keys[py.K_2]:
                attack = True
                yellow_hp -= 1


            # Uses stick
            elif keys[py.K_3] and stick:
                stick = False
                yellow_hp = 0
                attack = True
                
            # Uses arrow
            elif keys[py.K_4] and bow:
                bow = False
                yellow_hp = 0
                attack = True
                
            # Uses dagger
            elif keys[py.K_5] and dagger:
                dagger = False
                yellow_hp = 0
                attack = True
                

        # Yellow Attacks
        
        if attack == True:
            
            if timer <= 2:
                py.draw.rect(screen, 'pink', (0, 13*height/16, 7*width/10, height/8))
                
            else:
                py.draw.rect(screen, 'yellow', (0, 13*height/16, 7*width/10, height/8))
                py.draw.rect(screen, 'black', (0, 13*height/16, 7*width/10, height/8), 3)
                
                if 13*height/16 <= player_pos.y <= height:
                    attack = False
                    player_hp -= 1
                    timer = 100

            timer += 0.02
        
        if timer >= 3:
            attack = False
            timer = 0
            
            
        # Player Health Bar
        advice = font.render('health: ' + str(player_hp) + '/5', True, 'black')
        screen.blit(advice, (width/16, height/16))
        
        # Player Items
        advice = font.render(items, True, 'black')
        screen.blit(advice, (width/16, 3*height/16))
        
        # Enemy Health Bar
        inventory = font.render('health: ' + str(yellow_hp) + '/2', True, 'black')
        screen.blit(inventory, (13*width/16, height/16))
        
        # Updates comfort text
        if 0 <= stages <= 2:
            advice = font.render(negotiate[stages], True, 'black')
            screen.blit(advice, (width/8, height/2))
            
        
        # Updates Screen
        py.display.flip()
        
        # limits FPS to 60
        # dt is delta time in seconds since last frame, used for framerate
        dt = clock.tick(60) / 1000
        
        if yellow_hp == 0:
            colours['yellow'] = False
        
        
        if player_pos.x >= width-height/20 and attack == False:
            running = Red(width, height, coins, font, stick, player_hp, colours, string, arrow, bow, dagger)

    
    return(running)

# Green

In [141]:
def Green(width, height, coins, font, stick, player_hp, colours):

    
    # Collection of Variables
    dt = 0
    running = True
    player_pos = py.Vector2(width/8, 24*height/25)
    jumping = False
    attack = False
    block = False
    collect = False
    
    # Attack Timer
    timer = 0
    green_hp = 2
    
    
    # green Dialogue
    sickness = ['*Potion does nothing*', '*Square looks unfazed*', 'Green is weakened', '*Potion calms Green*  (You may now pass)']
    stages = -1
    
    while running:
        # Closes Program
        for event in py.event.get():
            if event.type == py.QUIT:
                return False
            
        # Player Options
        hints = 'PRESS 1 [potion], PRESS 2 [punch]'
        if stick == True:
            hints += ', PRESS 3 [stick]'
            
        # Generates inventory
        items = ''
        # Player's coins
        if coins != 0:
            items += (str(coins) + ' coin')
        
        if stick:
            items += (', stick')
        
        # Creating Background
        screen.fill("white")
        
        
        # Draws Player
        py.draw.circle(screen, "indigo", player_pos, height/23)
        py.draw.circle(screen, "black", player_pos, height/23, 3)
        
        # Draws green Square
        if green_hp != 0:
            py.draw.rect(screen, "green", (7*width/8, 11*height/12, height/12, height/12))
            py.draw.rect(screen, "black", (7*width/8, 11*height/12, height/12, height/12), 3)


        keys = py.key.get_pressed()
        
        # Player Jumps
        if keys[py.K_w] and player_pos.y == 24*height/25:

            jumping = True
            # Fixes a Teleportation Glitch
            dt = clock.tick(60) / 1000
            dt = 0
            
        # Circle is Being Pulled by Gravity
        if jumping and player_pos.y <= 24*height/25:
            player_pos.y -= (velocity*dt) + (-200*dt**2)
            velocity = velocity + (-400*dt)
        
        # Circle has Landed
        else:
            jumping = False
            player_pos.y = 24*height/25
            velocity = 3*height/5
            

        # Player Moves Horizontally
        if keys[py.K_a] and player_pos.x > 2*height/25:
            player_pos.x -= 300 * dt
            
        if keys[py.K_d]:
            if attack or green_hp != 0 and stages != 3:
                if player_pos.x < 13*width/20:
                    player_pos.x += 300 * dt
                    
                    
            elif player_pos.x < width-height/25:
                player_pos.x += 300 * dt

            
        # Player Engages With green Square
        if stages < 3 and green_hp != 0 and attack == False:
            hint = font.render(hints, True, 'black')
            screen.blit(hint, (width/8, height/4))
            
            # Use Potion on Green
            if keys[py.K_1]:
                stages += 1
                attack = True
                front = 3*width/5
                back = 3*width/4

                
            # Punches green
            elif keys[py.K_2]:
                attack = True
                front = 3*width/5
                back = 3*width/4
                block = True
                
                # Weak square takes damage
                if stages >= 2:
                    green_hp -= 1
                    block = False


            # Uses stick
            elif keys[py.K_3] and stick:
                stick = False
                attack = True
                front = 3*width/5
                back = 3*width/4
                coins += 1
                collect = True

        # green Attacks
        
        if attack == True:
            
            if timer <= 3:
                py.draw.polygon(screen, 'pink', [(front, 7*height/10), (front, height), (back, height)])
                
            else:
                front -= 6
                back -= 6
                py.draw.polygon(screen, 'green', [(front, 7*height/10), (front, height), (back, height)])
                py.draw.polygon(screen, 'black', [(front, 7*height/10), (front, height), (back, height)], 3)
                
                if front <= player_pos.x + height/23 <= front + 5 and player_pos.y + height/23 >= height - (back - player_pos.x) - height/23:
                    player_hp -= 1
                    attack = False
                    timer = 0
                    
                elif front <= player_pos.x - height/23 <= back and player_pos.y >= 9*height/10:
                    player_hp -= 1
                    attack = False
                    timer = 0

            timer += 0.06
        
            if 0 >= back:
                attack = False
                timer = 0
                block = False
            
            
        # Player Health Bar
        advice = font.render('health: ' + str(player_hp) + '/5', True, 'black')
        screen.blit(advice, (width/16, height/16))
        
        # Player Items
        advice = font.render(items, True, 'black')
        screen.blit(advice, (width/16, 3*height/16))
        
        # Enemy Health Bar
        inventory = font.render('health: ' + str(green_hp) + '/2', True, 'black')
        screen.blit(inventory, (13*width/16, height/16))
        
        # Updates comfort text
        if 0 <= stages <= 3:
            advice = font.render(sickness[stages], True, 'black')
            screen.blit(advice, (width/8, height/2))
            
        if block:
            strong = font.render('Green is uneffected', True, 'black')
            screen.blit(strong, (width/8, 2*height/5))
            if attack == False:
                block = False
            
        if collect:
            pickup = font.render('You found a coin', True, 'black')
            screen.blit(pickup, (width/8, 2*height/5))
            
        
        # Updates Screen
        py.display.flip()
        
        # limits FPS to 60
        # dt is delta time in seconds since last frame, used for framerate
        dt = clock.tick(60) / 1000
        
        if green_hp == 0:
            colours['green'] = False
            if collect == False:
                coins += 1
                collect = True
        
        if player_pos.x >= width-height/20 and attack == False:
            running = Yellow(width, height, coins, font, stick, player_hp, colours)

    
    return(running)

# Orange

In [142]:
def Orange(width, height, coins, font, stick, player_hp, colours):
    
    # Collection of Variables
    dt = 0
    running = True
    player_pos = py.Vector2(width/8, 24*height/25)
    jumping = False
    attack = False
    
    # Attack Timer
    timer = 0
    orange_hp = 4
    beam = 0
    
    
    # Orange Dialogue
    pride = ['*The square seems flustered*', '*orange winces*', 'orange seems flustered', '*orange gives up*  (You may now pass)']
    stages = -1
    
    while running:
        # Closes Program
        for event in py.event.get():
            if event.type == py.QUIT:
                return False
            
        # Player Options
        hints = 'PRESS 1 [mock], PRESS 2 [punch]'
        if stick == True:
            hints += ', PRESS 3 [stick]'
        
        # Creating Background
        screen.fill("white")
        
        # Generates inventory
        items = ''
        # Player's coins
        if coins != 0:
            items += (str(coins) + ' coin')
        
        if stick:
            items += (', stick')
        
        if beam == 4:
            beam = 3
        
        
        # Draws Player
        py.draw.circle(screen, "indigo", player_pos, height/23)
        py.draw.circle(screen, "black", player_pos, height/23, 3)
        
        # Draws orange Square
        if orange_hp != 0:
            py.draw.rect(screen, "orange", (7*width/8, 11*height/12, height/12, height/12))
            py.draw.rect(screen, "black", (7*width/8, 11*height/12, height/12, height/12), 3)


        keys = py.key.get_pressed()
        
        # Player Jumps
        if keys[py.K_w] and player_pos.y == 24*height/25:

            jumping = True
            # Fixes a Teleportation Glitch
            dt = clock.tick(60) / 1000
            dt = 0
            
        # Circle is Being Pulled by Gravity
        if jumping and player_pos.y <= 24*height/25:
            player_pos.y -= (velocity*dt) + (-200*dt**2)
            velocity = velocity + (-400*dt)
        
        # Circle has Landed
        else:
            jumping = False
            player_pos.y = 24*height/25
            velocity = 3*height/5
            

        # Player Moves Horizontally
        if keys[py.K_a] and player_pos.x > 2*height/25:
            player_pos.x -= 300 * dt
            
        if keys[py.K_d]:
            if attack or orange_hp != 0 and stages != 3:
                if player_pos.x < 13*width/20:
                    player_pos.x += 300 * dt
                    
                    
            elif player_pos.x < width-height/25:
                player_pos.x += 300 * dt
            
        # Player Engages With orange Square
        if stages < 3 and orange_hp != 0 and attack == False:
            hint = font.render(hints, True, 'black')
            screen.blit(hint, (width/8, height/4))
            
            # Mocks orange
            if keys[py.K_1]:
                stages += 1
                attack = True
                beam += 1
                spots = random.sample(range(1, 5), k=beam)
                
            # Punches orange
            elif keys[py.K_2]:
                orange_hp -= 1
                attack = True
                beam += 1
                spots = random.sample(range(1, 5), k=beam)

                
            # Uses stick
            elif keys[py.K_3] and stick:
                orange_hp = 0
                stick = False
                attack = True
                spots = random.sample(range(1, 5), k=beam)

        # Orange Attacks
        
        if attack == True:
            
            if timer <= 5:
                for x in spots:
                    
                    if x == 1:
                        py.draw.rect(screen, "pink", (width/18, 0, width/9, height))
                        
                    if x == 2:
                        py.draw.rect(screen, "pink", (4*width/18, 0, width/9, height))
                        
                    if x == 3:
                        py.draw.rect(screen, "pink", (7*width/18, 0, width/9, height))
                        
                    if x == 4:
                        py.draw.rect(screen, "pink", (10*width/18, 0, width/9, height))

            else:
                for x in spots:
                    
                    if x == 1:
                        py.draw.rect(screen, "orange", (width/18, 0, width/9, height))
                        if width/18 - height/30 <= player_pos.x <= 3*width/18 + height/30:
                            player_hp -= 1
                            attack = False
                            timer = 100
                        
                    if x == 2:
                        py.draw.rect(screen, "orange", (4*width/18, 0, width/9, height))
                        if 4*width/18 - height/30 <= player_pos.x <= 6*width/18 + height/30:
                            player_hp -= 1
                            attack = False
                            timer = 100
                        
                    if x == 3:
                        py.draw.rect(screen, "orange", (7*width/18, 0, width/9, height))
                        if 7*width/18 - height/30 <= player_pos.x <= 9*width/18 + height/30:
                            player_hp -= 1
                            attack = False
                            timer = 100
                        
                    if x == 4:
                        py.draw.rect(screen, "orange", (10*width/18, 0, width/9, height))
                        if 10*width/18 - height/30 <= player_pos.x <= 12*width/18 + height/30:
                            player_hp -= 1
                            attack = False
                            timer = 100

            timer += 0.06
        
        if timer >= 10:
            attack = False
            timer = 0
            
            
        # Player Health Bar
        advice = font.render('health: ' + str(player_hp) + '/5', True, 'black')
        screen.blit(advice, (width/16, height/16))
        
        # Player Items
        advice = font.render(items, True, 'black')
        screen.blit(advice, (width/16, 3*height/16))
        
        # Enemy Health Bar
        inventory = font.render('health: ' + str(orange_hp) + '/4', True, 'black')
        screen.blit(inventory, (13*width/16, height/16))
        
        # Updates comfort text
        if 0 <= stages <= 3:
            advice = font.render(pride[stages], True, 'black')
            screen.blit(advice, (width/8, height/2))
            
        
        # Updates Screen
        py.display.flip()
        
        # limits FPS to 60
        # dt is delta time in seconds since last frame, used for framerate
        dt = clock.tick(60) / 1000
        
        if orange_hp == 0:
            colours['orange'] = False
        
        
        if player_pos.x >= width-height/20 and attack == False:
            running = Green(width, height, coins, font, stick, player_hp, colours)


    
    return(running)

# Coin

In [143]:
def Coin(width, height, coins, font, stick, player_hp, blue_hp):
    
    # Collection of Variables
    dt = 0
    running = True
    player_pos = py.Vector2(width/8, 33*height/35)
    jumping = False
    collect = True

    
    while running:
        
        # Closes Program
        for event in py.event.get():
            if event.type == py.QUIT:
                return False
        
        # Generates inventory
        items = ''
        # Player's coins
        if coins != 0:
            items += (str(coins) + ' coin')
        
        if stick:
            items += (', stick')
            
            
        # Creating Background
        screen.fill("white")
        
        # Player Health Bar
        advice = font.render('health: ' + str(player_hp) + '/5', True, 'black')
        screen.blit(advice, (width/16, height/16))
        
        # Player Items
        advice = font.render(items, True, 'black')
        screen.blit(advice, (width/16, 3*height/16))
        
        
        # Draws Player
        py.draw.circle(screen, "indigo", player_pos, height/23)
        py.draw.circle(screen, "black", player_pos, height/23, 3)


        # Draws coin
        if collect:
            py.draw.circle(screen, "yellow", [9*width/10, 11*height/35], height/35, 0)
            py.draw.circle(screen, "black", [9*width/10, 11*height/35], height/35, 3)

        # Coin pickup
        if collect and 9*width/10-height/18 <= player_pos.x <= 9*width/10+height/18 and jumping == False:
            coins += 1
            collect = False
        
        
        # Draws platforms
        py.draw.rect(screen, "brown", (0, 34*height/35, width/4, height/35))
        py.draw.rect(screen, "brown", (3*width/8, 23*height/35, width/4, height/35))
        py.draw.rect(screen, "brown", (3*width/4, 12*height/35, width/4, height/35))
        
        py.draw.rect(screen, "black", (0, 34*height/35, width/4, height/35), 3)
        py.draw.rect(screen, "black", (3*width/8, 23*height/35, width/4, height/35), 3)
        py.draw.rect(screen, "black", (3*width/4, 12*height/35, width/4, height/35), 3)
        
        keys = py.key.get_pressed()
        

        # Player on platforms
        if 34*height/35 <= player_pos.y + height/23 <= height and 0 <= player_pos.x <= width/4:
            jumping = False
            
            
        elif 23*height/35 <= player_pos.y + height/23 <= 24*height/35 <= height and 3*width/8 <= player_pos.x <= 5*width/8:
            jumping = False

            
        elif 12*height/35 <= player_pos.y + height/23 <= 13*height/35 and 3*width/4 <= player_pos.x <= width:
            jumping = False

        # Fell off a platform
        elif jumping == False:
            jumping = True
            velocity = 0
            
        
        # Player Jumps
        if keys[py.K_w] and jumping == False:
            jumping = True
            player_pos.y -= height/16
            velocity = 3*height/5

            # Fixes a Teleportation Glitch
            dt = clock.tick(60) / 1000
            dt = 0    
        
        # Circle is Being Pulled by Gravity
        if jumping and player_pos.y <= height:
            player_pos.y -= (velocity*dt) + (-200*dt**2)
            velocity = velocity + (-400*dt)
            
        
        # Player Moves Horizontally
        if keys[py.K_a] and player_pos.x > height/25:
            player_pos.x -= 300 * dt
            
        if keys[py.K_d] and player_pos.x < width-height/25:
            player_pos.x += 300 * dt
            
        
        # Updates Screen
        py.display.flip()
        
        # limits FPS to 60
        # dt is delta time in seconds since last frame, used for framerate
        dt = clock.tick(60) / 1000
        
        if player_pos.y >= height:
            running = Blue(width, height, coins, font, stick, player_hp, blue_hp, 2, collect)


    return(running)

# BLUE

In [144]:
# Intializes player health
def Blue(width, height, coins, font, stick, player_hp, blue_hp, stages, up):
    
    # Collection of Variables
    dt = 0
    running = True
    player_pos = py.Vector2(width/8, 24*height/25)
    jumping = False
    attack = False
    
    # Attack Timer
    timer = 0
    
    
    # Alive Tracker
    colours = {'blue':True, 'orange':True, 'green': True, 'yellow':True, 'red':True, 'violet': True}
    
    # Blue Dialogue
    whining = ['*The square continues to cry*', '*Blue sniffles*', '*Blue stops crying*  (You may now pass)']
    
    while running:
        # Closes Program
        for event in py.event.get():
            if event.type == py.QUIT:
                return False
            
        # Player Options
        hints = 'PRESS 1 [comfort], PRESS 2 [punch]'
        if stick == True:
            hints += ', PRESS 3 [stick]'
            
        # Generates inventory
        items = ''
        # Player's coins
        if coins != 0:
            items += (str(coins) + ' coin')
        
        if stick:
            items += (', stick')
        
        # Creating Background
        screen.fill("white")
        
        # Player Health Bar
        advice = font.render('health: ' + str(player_hp) + '/5', True, 'black')
        screen.blit(advice, (width/16, height/16))
        
        # Player Items
        advice = font.render(items, True, 'black')
        screen.blit(advice, (width/16, 3*height/16))
        
        # Enemy Health Bar
        inventory = font.render('health: ' + str(blue_hp) + '/1', True, 'black')
        screen.blit(inventory, (13*width/16, height/16))
        
        # Draws Player
        py.draw.circle(screen, "indigo", player_pos, height/23)
        py.draw.circle(screen, "black", player_pos, height/23, 3)
        
        # Draws Blue Square
        if blue_hp != 0:
            py.draw.rect(screen, "blue", (7*width/8, 11*height/12, height/12, height/12))
            py.draw.rect(screen, "black", (7*width/8, 11*height/12, height/12, height/12), 3)
            
        # Draws platforms
        py.draw.rect(screen, "brown", (0, 0, width/4, height/35))
        py.draw.rect(screen, "brown", (3*width/8, 12*height/35, width/4, height/35))
        py.draw.rect(screen, "brown", (3*width/4, 23*height/35, width/4, height/35))
        
        py.draw.rect(screen, "black", (0, 0, width/4, height/35), 3)
        py.draw.rect(screen, "black", (3*width/8, 12*height/35, width/4, height/35), 3)
        py.draw.rect(screen, "black", (3*width/4, 23*height/35, width/4, height/35), 3)

        keys = py.key.get_pressed()
            

        # Player on platforms
        if 23*height/35 <= player_pos.y + height/24 <= 24*height/35 and 3*width/4 <= player_pos.x <= width:
            jumping = False
            
            
            
        elif 12*height/35 <= player_pos.y + height/23 <= 13*height/35 <= height and 3*width/8 <= player_pos.x <= 5*width/8:
            jumping = False
            
        
        elif player_pos.y >= 24*height/25:
            jumping = False
            player_pos.y = 24*height/25
            

        # Fell off a platform
        elif jumping == False:
            jumping = True
            velocity = 0
            
        
        # Player Jumps
        if keys[py.K_w] and jumping == False:
            jumping = True
            player_pos.y -= height/16
            velocity = 3*height/5

            # Fixes a Teleportation Glitch
            dt = clock.tick(60) / 1000
            dt = 0    
        
        # Circle is Being Pulled by Gravity
        if jumping and player_pos.y <= height:
            player_pos.y -= (velocity*dt) + (-200*dt**2)
            velocity = velocity + (-400*dt)

        # Player Moves Horizontally
        if keys[py.K_a] and player_pos.x > height/25:
            player_pos.x -= 300 * dt
            

        if keys[py.K_d]:
            if attack or blue_hp != 0 and stages != 2:
                if player_pos.x < 5*width/8:
                    player_pos.x += 300 * dt
            elif player_pos.x < width-height/25:
                player_pos.x += 300 * dt
    
            
        # Player Engages With Blue Square
        if stages < 2 and blue_hp != 0 and attack == False:
            hint = font.render(hints, True, 'black')
            screen.blit(hint, (width/8, height/4))
            
            # Comforts Blue
            if keys[py.K_1]:
                stages += 1
                attack = True
                
            # Punches Blue
            elif keys[py.K_2]:
                blue_hp -= 1
                attack = True
                
            # Uses stick
            elif keys[py.K_3] and stick:
                blue_hp = 0
                stick = False
                attack = True
            

        # Blue Attacks
        
        if attack == True:
            if timer <= 6:
                py.draw.rect(screen, "pink", (0, 34*height/35, width, height/35))

            else:
                py.draw.rect(screen, "blue", (0, 34*height/35, width, height/35))
                if player_pos.y == 24*height/25:
                    player_hp -= 1
                    attack = False
                    timer = 20

            timer += 0.06
        
        if timer >= 10:
            attack = False
            timer = 0
            
            
        
        # Updates comfort text
        if 0 <= stages <= 2:
            advice = font.render(whining[stages], True, 'black')
            screen.blit(advice, (width/8, height/2))
            
        
        # Updates Screen
        py.display.flip()
        
        # limits FPS to 60
        # dt is delta time in seconds since last frame, used for framerate
        dt = clock.tick(60) / 1000
        
        if blue_hp == 0:
            colours['blue'] = False
        
        if player_pos.x >= width-height/20 and attack == False:
                running = Orange(width, height, coins, font, stick, player_hp, colours)
            
        if 0 <= player_pos.x <= width/4 and player_pos.y <= 0 and up == True:
            running = Coin(width, height, coins, font, stick, player_hp, blue_hp)

    
    return(running)

# Gray

In [145]:
def Gray(width, height, coins, font):
    
    # Collection of Variables
    dt = 0
    running = True
    player_pos = py.Vector2(7*width/8, 24*height/25)
    jumping = False
    
    
    # Gray Dialogue
    dialogue = ['Hello Adventurer', 'You must unite the colours', 'Travel east-ward and you will encounter them', 'Remember, pink always comes before an attack', 'May this stick help you', 'Good luck Adventurer']
    option = -1
    stick = False
    
    # Gameplay
    while running:
        # Closes Program
        for event in py.event.get():
            if event.type == py.QUIT:
                return False
            
            
        # Generates inventory
        items = ''
        # Player's coins
        if coins != 0:
            items += (str(coins) + ' coin')
        
        if stick:
            items += (', stick')
        
        # Creating Background
        screen.fill("white")
        
        # Health Bar
        advice = font.render('health: 5/5', True, 'black')
        screen.blit(advice, (width/16, height/16))
        
        # Player Items
        inventory = font.render(items, True, 'black')
        screen.blit(inventory, (width/16, 3*height/16))
        
        # Draws Player
        py.draw.circle(screen, "indigo", player_pos, height/23, 0)
        py.draw.circle(screen, "black", player_pos, height/23, 3)
        
        # Draws Gray Square
        py.draw.rect(screen, "grey", (width/5, 11*height/12, height/12, height/12))
        py.draw.rect(screen, "black", (width/5, 11*height/12, height/12, height/12), 3)
                

        keys = py.key.get_pressed()
        
        # Player Jumps
        if keys[py.K_w] and player_pos.y == 24*height/25:

            jumping = True
            # Fixes a Teleportation Glitch
            dt = clock.tick(60) / 1000
            dt = 0
            
        # Circle is Being Pulled by Gravity
        if jumping and player_pos.y <= 24*height/25:
            player_pos.y -= (velocity*dt) + (-200*dt**2)
            velocity = velocity + (-400*dt)
        
        # Circle has Landed
        else:
            jumping = False
            player_pos.y = 24*height/25
            velocity = 3*height/5
            

        # Player Moves Horizontally
        if keys[py.K_a] and player_pos.x > height/25:
            player_pos.x -= 300 * dt
            
        if keys[py.K_d] and player_pos.x < width-height/25:
            player_pos.x += 300 * dt
    
            
        # Player Engages With Gray Square
        if player_pos.x < width/3 and option <= 5:
            hint = font.render('(do not hold) PRESS 1 [talk]', True, 'black')
            screen.blit(hint, (width/8, height/3))
            
            if keys[py.K_1]:
                option += 1
                time.sleep(0.3)
        
        if 0 <= option <= 5:
            advice = font.render(dialogue[option], True, 'black')
            screen.blit(advice, (width/8, height/2))
            
        if option >= 4:
            stick = True
        
            
        # Updates Screen
        py.display.flip()
        
        # limits FPS to 60
        # dt is delta time in seconds since last frame, used for framerate
        dt = clock.tick(60) / 1000
        
        
        if player_pos.x >= width-height/20:
            running = Blue(width, height, coins, font, stick, 5, 1, -1, True)


    
    return(running)

In [146]:
# Basic Setup
py.init()
py.font.init()

# FPS set-up
clock = py.time.Clock()

# Screen Parameters
width = 1000
height = 700

# Creating Font
font = py.font.SysFont('timesnewroman.ttf', int(width/30))

# Sets Up Window
screen = py.display.set_mode((width, height))
game1 = True


def Start():
    
    dt = 0
    running = True
    player_pos = py.Vector2(width/2, 17*height/18)

    # Displaying Text
    text = font.render('Turn Back', True, 'black')
    
    # Text
    py.display.set_caption('Colours')
    
    
    
    reveal = False
    jumping = False
    pickup = False
    coins = 0
    
    while running:
        # Closes Program
        for event in py.event.get():
            if event.type == py.QUIT:
                running = False
        
        # Generates inventory
        items = ''
        # Player's coins
        if coins != 0:
            items += (str(coins) + ' coin')
        
        # Creating Visuals
        screen.fill("white")
        py.draw.circle(screen, "indigo", player_pos, height/23, 0)
        py.draw.circle(screen, "black", player_pos, height/23, 3)
        
        # Health Bar
        advice = font.render('health: 5/5', True, 'black')
        screen.blit(advice, (width/16, height/16))
        
        if player_pos.x >= width-height/20:
            reveal = True
            
        # Player Items
        inventory = font.render(items, True, 'black')
        screen.blit(inventory, (width/16, 3*height/16))
        
    
        # Displaying coin
        if reveal == True and pickup == False:
            py.draw.circle(screen, "yellow", [width/10, 34*height/35], height/35, 0)
            py.draw.circle(screen, "black", [width/10, 34*height/35], height/35, 3)
            
                
        if reveal == True and width/10-height/18 <= player_pos.x <= width/10+height/18 and jumping == False:
            reveal = False
            coins = 1
            pickup = True
                

        keys = py.key.get_pressed()
        # Player Jumps

        if keys[py.K_w] and player_pos.y == 24*height/25:

            jumping = True
            
            dt = clock.tick(60) / 1000
            dt = 0
            
        if jumping and player_pos.y <= 24*height/25:
            player_pos.y -= (velocity*dt) + (-200*dt**2)
            velocity = velocity + (-400*dt)
            
        else:
            jumping = False
            player_pos.y = 24*height/25
            velocity = 3*height/5
            

        # Player Moves Horizontally
        if keys[py.K_a] and player_pos.x > height/25:
            player_pos.x -= 300 * dt
            
        if keys[py.K_d] and player_pos.x < width-height/25:
            player_pos.x += 300 * dt
    
            
        py.display.flip()
        
        # limits FPS to 60
        # dt is delta time in seconds since last frame, used for framerate
        dt = clock.tick(60) / 1000
        
        if player_pos.x <= height/20:
            running = Gray(width, height, coins, font)
            
    return (running)
        

if game1 == True:
    Start()
    game1 == False
    
py.quit()